In [2]:
# import pandas as pd
# df = pd.read_csv("tat_qa.csv", on_bad_lines='skip')
# # 使用 str.split 方法將列中的每個值按 '|' 分割，expand=True 會將分割後的結果展開為多列
# df_split = df.iloc[:, 0].str.split('|', expand=True)

# # 重新設置列名（如果需要）
# df_split.columns = ['doc_name', 'q_uid', 'question', 'answer', 'answer_scale', 'answer_type']

# # 保存為新的 CSV 文件
# df_split.to_csv('tat_qa_split.csv', index=False)

# # 打印結果以驗證
# print(df_split.head())

# # 8966 rows × 6 columns,但是實際上應該有14703個檔案，所以有6000個bad line，良率偏差（？

In [3]:
# df_split

In [4]:
#GPT revised

import pandas as pd

# 定义一个函数来安全地解析每一行
def safe_parse_line(line):
    parts = line.split('|')
    if len(parts) == 5:  # 检查行是否正确地分成了5部分
        return parts
    else:
        # 如果行的格式不正确，可以选择跳过或处理它
        return None

# 逐行读取文件并手动处理
rows = []
with open('csv/feta_qa.csv', 'r', encoding='utf-8') as file:
    for line in file:
        parsed_line = safe_parse_line(line.strip())
        if parsed_line:
            rows.append(parsed_line)

# 将处理后的行转换为 DataFrame|question|answer|doc_url
df_split = pd.DataFrame(rows, columns=['doc_name', 'q_uid', 'question', 'answer', 'doc_url'])

# 保存为新的 CSV 文件
df_split.to_csv('csv/feta_qa_split.csv', index=False)

# 打印结果以验证
print(df_split.head())

# 打印总行数以验证
print(f"Total rows: {df_split.shape[0]}")


                  doc_name  q_uid  \
0                 doc_name  q_uid   
1        Ben Platt (actor)  14717   
2        Ben Platt (actor)  16743   
3  List of French monarchs  19148   
4  List of French monarchs  18732   

                                            question  \
0                                           question   
1  When and in what play did Platt appear at the ...   
2  Which role did Ben Platt play at the Second St...   
3  What year did Philip I the Amorous become King...   
4  During which years was Henry I king of the fra...   

                                              answer  \
0                                             answer   
1  In 2016 and 2017, Platt played in Dear Evan Ha...   
2  In 2016, Platt played the role of Evan Hansen ...   
3  Philip I the Amorous, was King of the Franks f...   
4  Henry I was King of the Franks from 1031 to 10...   

                                             doc_url  
0                                            doc

In [5]:
grouped_df = df_split.groupby('doc_name').size().reset_index(name='count')
grouped_df = grouped_df.sort_values(by='count', ascending=False)
grouped_df

,doc_name,count
439,List of Chief Ministers of Punjab (India),4
448,List of French monarchs,4
437,List of Chief Ministers of Nagaland,4
399,Koenigsegg Agera,4
33,2018 Tour de France,4
...,...,...
309,Ian McDiarmid,1
310,Ice hockey at the 2010 Winter Olympics – Men's...,1
311,If I Had a Heart,1
312,Indian cricket team in England in 1932,1


In [6]:
import os
folder_path = './wiki_docs'  # 請替換成你實際的資料夾路徑

def get_file_size(doc_name):
    file_path = os.path.join(folder_path, doc_name+".pdf")
    if os.path.exists(file_path):
        return os.path.getsize(file_path)
    return None

# 新增檔案大小欄位
grouped_df['file_size'] = grouped_df['doc_name'].apply(get_file_size)

# 計算檔案大小除以 count 的比值，一個count要花多少大小才能夠查詢到，越小越好
grouped_df['size_per_count'] = grouped_df['file_size'] / grouped_df['count'] 

# # 刪除包含 None 的行 (也就是含有 'pdf' 的檔案名)
grouped_df = grouped_df.dropna(subset=['file_size'])

# 按 count 列進行降序排序
grouped_df = grouped_df.sort_values(by='size_per_count', ascending=True)

grouped_df

,doc_name,count,file_size,size_per_count
437,List of Chief Ministers of Nagaland,4,278611.0,6.965275e+04
461,List of NASCAR drivers who have won in each of...,1,70281.0,7.028100e+04
202,Eleanor Tomlinson,3,340322.0,1.134407e+05
46,AirPort Time Capsule,3,342353.0,1.141177e+05
327,James Monroe Iglehart,2,246809.0,1.234045e+05
...,...,...,...,...
447,List of Formula One driver records,1,5468088.0,5.468088e+06
203,Electric car use by country,1,6083798.0,6.083798e+06
458,List of Marvel Cinematic Universe films,1,6634187.0,6.634187e+06
523,List of iOS devices,1,6874768.0,6.874768e+06


In [7]:
# 選取前10大的檔案
top_30 = grouped_df.head(30)

# 計算 count 總數加總
total_count_sum = top_30['count'].sum()

# 找出包含 'pdf' 的文件名列表
pdf_list = top_30['doc_name'].tolist()

# 輸出結果
print("前10大的檔案 count 總數加總:", total_count_sum)
print("PDF 文件列表:", pdf_list)


前10大的檔案 count 總數加總: 67
PDF 文件列表: ['List of Chief Ministers of Nagaland', 'List of NASCAR drivers who have won in each of top three series', 'Eleanor Tomlinson', 'AirPort Time Capsule', 'James Monroe Iglehart', 'Emma Chambers', 'List of Chief Ministers of Punjab (India)', 'Tom Hughes (actor)', 'Dane DeHaan', 'List of Doraemon films', 'List of actors nominated for two Academy Awards in the same year', 'Dheeraj Dhoopar', 'List of Prime Ministers of Tanzania', 'Kevin Peter Hall', 'Northstar engine series', 'List of governors of Bengal', 'List of Speakers of the National Assembly of Guyana', 'Koenigsegg Agera', 'Home Alone: Original Motion Picture Soundtrack', 'List of undefeated boxing world champions', 'Chloe Lanier', 'Victoria Hamilton', 'Kristian Schmid', 'Lindsay and Sidney Greenbush', 'Kathryn Beaumont', 'Jesse Plemons', 'Taissa Farmiga', 'List of Canadian islands by area', 'Major League Baseball consecutive games played streaks', 'List of U.S. state shells']


In [8]:
pdf_list

['List of Chief Ministers of Nagaland',
 'List of NASCAR drivers who have won in each of top three series',
 'Eleanor Tomlinson',
 'AirPort Time Capsule',
 'James Monroe Iglehart',
 'Emma Chambers',
 'List of Chief Ministers of Punjab (India)',
 'Tom Hughes (actor)',
 'Dane DeHaan',
 'List of Doraemon films',
 'List of actors nominated for two Academy Awards in the same year',
 'Dheeraj Dhoopar',
 'List of Prime Ministers of Tanzania',
 'Kevin Peter Hall',
 'Northstar engine series',
 'List of governors of Bengal',
 'List of Speakers of the National Assembly of Guyana',
 'Koenigsegg Agera',
 'Home Alone: Original Motion Picture Soundtrack',
 'List of undefeated boxing world champions',
 'Chloe Lanier',
 'Victoria Hamilton',
 'Kristian Schmid',
 'Lindsay and Sidney Greenbush',
 'Kathryn Beaumont',
 'Jesse Plemons',
 'Taissa Farmiga',
 'List of Canadian islands by area',
 'Major League Baseball consecutive games played streaks',
 'List of U.S. state shells']

In [1]:
#修改paddle的原始碼，把tools前面增加paddle.
from run_parse import run_parse
from chunking import merge_chunk_content_new
import time
import json
results = {}
api_key = "sk-None-38wdILNqVMe1RLcurKHfT3BlbkFJALAog4oQSCSmWL3fe8ie"
name = "臺灣大學測試.pdf"
# chinese_cht
txt_pages, GPT_CALL_COUNT,image_dict = run_parse(name, False, "chinese_cht", api_key, finance=False, page=False)
# with open("咒術data＿new.json", "w", encoding="utf-8") as file:
#     json.dump(txt_pages, file, ensure_ascii=False, indent=4)
    # results[i] = {
    #     "path": path,
    #     "image_dict": image_dict,
    #     "start_time": start_time,
    #     "end_time": end_time,
    #     "time_elapsed": elapsed_time,
    #     "GPT_CALL_COUNT": GPT_CALL_COUNT,
    #     "NOTSAMELENGTH" : NOTSAMELENGTH,
    # }
    
    # print(results[i])
    # break

# # 生成文件名，包含当前时间戳
# timestamp = time.strftime("%Y%m%d_%H%M%S")
# filename = "0826_wiki30.json"

# # 将当前的结果保存为JSON文件
# json_data = json.dumps(results[i], indent=4)
# with open(filename, "w") as json_file:
#     json_file.write(json_data)


2024-09-09 13:28:12,968 - INFO - Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-09-09 13:28:12,968 - INFO - NumExpr defaulting to 8 threads.



PDF頁數: 2
parse page: 0
parse page: 1
['0_0.png', '0_1.png', '0_2.png', '1_0.png', '1_1.png', '1_2.png', '1_3.png']
[[(35.25, 525.0, 240.0, 640.5), (363.0, 27.75, 559.5, 524.25), (403.5, 535.5, 559.5, 651.75)], [(35.25, 27.75, 312.0, 814.5), (328.5, 33.75, 559.5, 208.5), (403.5, 229.5, 559.5, 354.0), (388.5, 375.75, 559.5, 552.0)]]
['0_0.png', '0_1.png', '0_2.png', '1_0.png', '1_1.png', '1_2.png', '1_3.png']
Parse_pdf抓到的內容： [(35.25, 525.0, 240.0, 640.5), (363.0, 27.75, 559.5, 524.25), (403.5, 535.5, 559.5, 651.75)]
pymupdf抓到的表格： [{'bbox': (35.5625, 524.7321166992188, 239.6875, 640.21875), 'rows': 5, 'columns': 3}]
pymupdf抓到的圖片： [{'number': 16, 'bbox': (406.5, 538.5, 556.5, 615.75), 'transform': (150.0, 0.0, -0.0, 77.25, 406.5, 538.5), 'width': 400, 'height': 206, 'colorspace': 3, 'cs-name': 'ICCBased(RGB,Google/Skia/7C5FA2151397474A0486BBCC83733D59)', 'xres': 96, 'yres': 96, 'bpc': 8, 'size': 28131}]
Parse_pdf抓到的內容： [(35.25, 27.75, 312.0, 814.5), (328.5, 33.75, 559.5, 208.5), (403.5, 2

In [2]:
import json
with open("舞台劇data.json", "w", encoding="utf-8") as file:
    json.dump(txt_pages, file, ensure_ascii=False, indent=4)


In [2]:
from chunking import merge_chunk_content_new

ans = merge_chunk_content_new(txt_pages,chunk_size=400,merge_tables = True)


content 1:
(75, 2113, 'table', '<table border="1">    <tr>        <th>角色名稱</th>        <th>演員</th>        <th>第1彈</th>        <th>第2彈</th>        <th>備註</th>    </tr>    <tr>        <td rowspan="6">主要角色</td>        <td>虎杖悠仁</td>        <td>佐藤流司</td>        <td></td>        <td></td>    </tr>    <tr>        <td>伏黑惠</td>        <td>泰江和明</td>        <td></td>        <td></td>    </tr>    <tr>        <td>釘崎野薔薇</td>        <td>豐原江理佳</td>        <td>山口乃乃華</td>        <td></td>    </tr>    <tr>        <td>五條悟</td>        <td>三浦涼介</td>        <td></td>        <td></td>    </tr>    <tr>        <td>兩面宿儺</td>        <td>五十嵐拓人</td>        <td>不適用</td>        <td></td>    </tr>    <tr>        <td colspan="2" rowspan="8">東京咒術高專</td>        <td>禪院真希</td>        <td>高月彩良</td>        <td></td>    </tr>    <tr>        <td>狗卷棘</td>        <td>定本楓馬</td>        <td></td>        <td></td>    </tr>    <tr>        <td>貓熊</td>        <td>寺山武志</td>        <td></td>        <td></td>    </tr>    <tr>        <td>夜

In [1]:
from chunking import merge_chunk_content_new, split_text
import json
# 從文件讀取
with open("data.json", "r", encoding="utf-8") as file:
    txt_pages = json.load(file)

# 從文件讀取
merged_chunks, chunk_page = merge_chunk_content_new(txt_pages, chunk_size=500, merge_tables=True)

(2680, 3602, 'table', '<table>  <tr>    <th rowspan="2">咒術迴戰</th>    <th>Col2</th>  </tr>  <tr>    <td></td>  </tr>  <tr>    <td colspan="2">台灣中文版漫畫單行本第1卷封面</td>  </tr>  <tr>    <td colspan="2">呪術廻戦 Jujutsu Kaisen</td>  </tr>  <tr>    <td>假名</td>    <td>じゅじゅつかいせん</td>  </tr>  <tr>    <td>類型</td>    <td>少年漫畫、黑暗奇幻、校園、戰鬥</td>  </tr>  <tr>    <td>正式譯名</td>    <td>[註 1] 咒術迴戰 咒术回战</td>  </tr>  <tr>    <td>漫畫</td>    <td></td>  </tr>  <tr>    <td>作者</td>    <td>芥見下下</td>  </tr>  <tr>    <td>出版社</td>    <td>集英社 文化傳信 東立出版社 柒海圖書（代理）、西泠印社出版社（出版）</td>  </tr>  <tr>    <td>連載雜誌</td>    <td>週刊少年Jump 寶島少年</td>  </tr>  <tr>    <td>叢書</td>    <td>JUMP COMICS CITICOMICS</td>  </tr>  <tr>    <td>連載網站</td>    <td>Bilibili</td>  </tr>  <tr>    <td>連載期間</td>    <td>2018年14號—2024年44號</td>  </tr>  <tr>    <td>發表期間</td>    <td>2018年3月5日—2024年9月30日</td>  </tr>  <tr>    <td>冊數</td>    <td>27卷（2024年7月） 26卷（2024年5月） 24卷（2024年2月） 10卷（2022年9月）</td>  </tr></table>')
(4009, 4569, 'table', '<table>    <tr>        <td ro

In [4]:
txt_pages[5]["text"]

'2024/8/28 下午2:27 管中閔 - 維基百科，自由的百科全書|中華民國政府職務|Col2|Col3||---|---|---||行政院||||前任： 尹啟銘|行政院經濟建設委員會主任委員 第十六任 2013年2月18日－2014年1月22日|改制為國發會||首任|國家發展委員會主任委員 第一任 2014年1月22日－2015年2月4日|繼任： 杜紫軍||教育職務|Col2|Col3||---|---|---||國立臺灣大學||||前任： 郭大維（代理） 正任：楊泮池|國立臺灣大學校長 第十二任 2019年1月8日－2023年1月8日|繼任： 陳文章|**教育職務**[取自「管中閔&oldid=83628827」](管中閔 6/6-----single table:```html\n<table border="1" cellpadding="5" cellspacing="0">\n    <thead>\n        <tr>\n            <th colspan="3" style="text-align: center;">中華民國政府職務</th>\n        </tr>\n    </thead>\n    <tbody>\n        <tr>\n            <td rowspan="2">行政院</td>\n            <td colspan="2">前任： 尹啟銘<br>行政院經濟建設委員會主任委員 第十六任 2013年2月18日－2014年1月22日</td>\n        </tr>\n        <tr>\n            <td colspan="2">首任<br>國家發展委員會主任委員 第一任 2014年1月22日－2015年2月4日<br>繼任： 杜紫軍</td>\n        </tr>\n        <tr>\n            <td colspan="3">教育職務</td>\n        </tr>\n        <tr>\n            <td rowspan="2">國立臺灣大學</td>\n            <td colspan="2">前任：郭大維（代理）<br>正任：楊洋池<br>2019年1月8日－2

In [5]:
chunk_page[-1]["chunk"]

'<table border="1" cellpadding="5" cellspacing="0">\n    <thead>\n        <tr>\n            <th colspan="3" style="text-align: center;">中華民國政府職務</th>\n        </tr>\n    </thead>\n    <tbody>\n        <tr>\n            <td rowspan="2">行政院</td>\n            <td colspan="2">前任： 尹啟銘<br>行政院經濟建設委員會主任委員 第十六任 2013年2月18日－2014年1月22日</td>\n        </tr>\n        <tr>\n            <td colspan="2">首任<br>國家發展委員會主任委員 第一任 2014年1月22日－2015年2月4日<br>繼任： 杜紫軍</td>\n        </tr>\n        <tr>\n            <td colspan="3">教育職務</td>\n        </tr>\n        <tr>\n            <td rowspan="2">國立臺灣大學</td>\n            <td colspan="2">前任：郭大維（代理）<br>正任：楊洋池<br>2019年1月8日－2023年1月8日</td>\n        </tr>\n        <tr>\n            <td colspan="2">教育職務</td>\n        </tr>\n    </tbody>\n</table>```\nend of single table'

In [ ]:
#======以下為非必要資訊，僅供參考======

In [2]:
#wiki也做類似的事情，得到另一個informative的資料集
#feta_qa
rows = []
with open('csv/feta_qa.csv', 'r', encoding='utf-8') as file:
    for line in file:
        parsed_line = safe_parse_line(line.strip())
        if parsed_line:
            rows.append(parsed_line)
print(rows)
# 将处理后的行转换为 DataFrame
feta_df_split = pd.DataFrame(rows, columns=['doc_name', 'q_uid', 'question', 'answer', 'doc_url'])

# 保存为新的 CSV 文件
feta_df_split.to_csv('csv/feta_df_split.csv', index=False)

# 打印结果以验证
print(feta_df_split.head())

# 打印总行数以验证
print(f"Total rows: {feta_df_split.shape[0]}")


NameError: name 'safe_parse_line' is not defined

In [27]:
#修改paddle的原始碼，把tools前面增加paddle.
from run_parse import run_parse
import time
pathl = []
image_dictl = []
textl = []
timel = []
#開始時間:12.26，開始價格：未知
api_key = "sk-None-38wdILNqVMe1RLcurKHfT3BlbkFJALAog4oQSCSmWL3fe8ie"
pdfs = ["gaslog-ltd_2019",'stmicroelectronics_2019', 'teradyne-inc_2019', 'vmware-inc_2019', 'leidos-holdings_2019', 'finjan-holding-inc_2019', 'cornerstone-ondemand-inc_2019', 'mitek-systems_2019', 'oracle-corporation_2019', 'plexus-corp_2019']
for i in pdfs:
    start_time = time.time()
    name = "tat_docs/" + i + ".pdf"
    print(name)
    path,image_dict = run_parse(name,api_key,finance=True,page=True)
    image_dictl.append(image_dict)
    pathl.append(path)
    end_time = time.time()
    print(end_time-start_time)
    timel.append()
    #11:45左右，處理倒地１６３頁的時後當機，沒辦法看到錯誤訊息．

tat_docs/gaslog-ltd_2019.pdf
start processing:  tat_docs/gaslog-ltd_2019.pdf
parse page: 0
parse page: 1
parse page: 2
parse page: 3
parse page: 4
parse page: 5
parse page: 6
parse page: 7
parse page: 8
parse page: 9
parse page: 10
parse page: 11
parse page: 12
parse page: 13
parse page: 14
parse page: 15
parse page: 16
parse page: 17
parse page: 18
parse page: 19
parse page: 20
parse page: 21
parse page: 22
parse page: 23
parse page: 24
parse page: 25
parse page: 26
parse page: 27
parse page: 28
parse page: 29
parse page: 30
parse page: 31
parse page: 32
parse page: 33
parse page: 34
parse page: 35
parse page: 36
parse page: 37
parse page: 38
parse page: 39
parse page: 40
parse page: 41
parse page: 42
parse page: 43
parse page: 44
parse page: 45
parse page: 46
parse page: 47
parse page: 48
parse page: 49
parse page: 50
parse page: 51
parse page: 52
parse page: 53
parse page: 54
parse page: 55
parse page: 56
parse page: 57
parse page: 58
parse page: 59
parse page: 60
parse page: 61
par

ValueError: not enough values to unpack (expected 3, got 2)

In [2]:
from GeneralAgent import Agent
role_prompt = """你現在是一位專注於製作HTML表格的工程師，你的任務是要畫出一個可以顯示的表格。
"""
local_prompt = """你現在是一個製作HTML表格的工程師，你的任務是將圖片中表格的架構與文字轉成HTML表格，你必須要做到:
1.請判斷該圖片中的上下文與表格之間是否有關聯，如果無關，可以請你無視。
2.請專注在圖片表格的結構，完整的表現出原本的架構。
3.請專注在圖片文字，盡量預測正確。
4.請注意合併儲存格，讓結構完整。   
5.請你注意表格生成的合理性，並判斷表格位置是否正確。
6. 請不要輸出除了html格式以外的任何內容，例如「以下是輸出的檔案內容」等等
"""
agent = Agent(role=role_prompt, api_key="sk-None-38wdILNqVMe1RLcurKHfT3BlbkFJALAog4oQSCSmWL3fe8ie", base_url=None, model="gpt-4o", disable_python_run=False)
local_prompt = local_prompt
content = agent.run([local_prompt, {'image':'pic/2_0.png'}])

In [3]:
content

'```html\n<div>\n    <p>第三條 數位存款帳戶交易限制</p>\n    <p>\n        一、數位存款帳戶之金融卡提款預設限額如下，除本約定條款另有約定者外，數位存款帳戶之金融卡提款限額，悉依綜合約定書、併同金融卡提款交易客戶約定條款等相關約定辦理：\n    </p>\n</div>\n<table border="1">\n    <tr>\n        <th>帳戶驗證方式</th>\n        <th>臺幣提款限額</th>\n    </tr>\n    <tr>\n        <td>視訊及自然人憑證、分行升級(1-1 類)</td>\n        <td>每帳戶每日最高新臺幣20萬元。</td>\n    </tr>\n    <tr>\n        <td rowspan="3">既有戶加辦(2 類)<br>自然人憑證(1-2 類)</td>\n        <td>1. 「消費扣款」及「金融卡付款-跨境線上交易」每筆最高新臺幣1萬元。</td>\n    </tr>\n    <tr>\n        <td>2. 每帳戶每日及每月最高新臺幣10萬元。</td>\n    </tr>\n    <tr>\n        <td>3. 申請人不得自行於「網銀 App」調整該帳戶之金融卡提款限額。</td>\n    </tr>\n    <tr>\n        <td>他行帳戶(3-1 類)</td>\n        <td></td>\n    </tr>\n    <tr>\n        <td>卡友加開(3-2 類)</td>\n        <td></td>\n    </tr>\n    <tr>\n        <td colspan="2">\n            備註：上述提款每日及每月限額含金融卡臺幣提款、無卡提款、扣領新臺幣帳戶提領外幣現鈔、消費扣款、金融卡付款-跨境線上交易。\n        </td>\n    </tr>\n</table>\n```\n'

In [4]:
import pymupdf
doc = pymupdf.open("課表.pdf")
page = doc[0]
paths = page.get_drawings()  # extract existing drawings
# this is a list of "paths", which can directly be drawn again using Shape
# -------------------------------------------------------------------------

# define some output page with the same dimensions
outpdf = pymupdf.open()
outpage = outpdf.new_page(width=page.rect.width, height=page.rect.height)
shape = outpage.new_shape()  # make a drawing canvas for the output page
# --------------------------------------
# loop through the paths and draw them
# --------------------------------------
for path in paths:
    # ------------------------------------
    # draw each entry of the 'items' list
    # ------------------------------------
    for item in path["items"]:  # these are the draw commands
        if item[0] == "l":  # line
            shape.draw_line(item[1], item[2])
        elif item[0] == "re":  # rectangle
            shape.draw_rect(item[1])
        #elif item[0] == "qu":  # quad
        #    shape.draw_quad(item[1])
        #elif item[0] == "c":  # curve
        #    shape.draw_bezier(item[1], item[2], item[3], item[4])
        else:
            raise ValueError("unhandled drawing", item)
    # ------------------------------------------------------
    # all items are drawn, now apply the common properties
    # to finish the path
    # ------------------------------------------------------
    shape.finish(
        fill=path["fill"],  # fill color
        color=path["color"],  # line color
        
        )
# all paths processed - commit the shape to its page
shape.commit()
outpdf.save("drawings-page-0.pdf")

In [18]:
import shapely.geometry as sg
import fitz
from shapely.validation import explain_validity
import os
pdf_document = fitz.open('課表.pdf')
for ind,i in enumerate(pdf_document):
    rect_list = [sg.box(*drawing['rect']) for drawing in i.get_drawings()]
    print(rect_list)
    merged_rects = [rect for rect in rect_list if explain_validity(rect) == 'Valid Geometry']
    print(merged_rects)
    rects  = [rect.bounds for rect in merged_rects]
    for index, rect in enumerate(rects):
        fitz_rect = fitz.Rect(rect)

        # 保存页面为图片
        pix = page.get_pixmap(clip=fitz_rect, matrix=fitz.Matrix(4, 4))
        name = f'{ind}_{index}.png'
        pix.save(os.path.join('./', name))



[<POLYGON ((596 0, 596 842, 0 842, 0 0, 596 0))>, <POLYGON ((138.009 107.193, 138.009 131.068, 58.193 131.068, 58.193 107.193,...>, <POLYGON ((217.824 107.193, 217.824 131.068, 138.009 131.068, 138.009 107.19...>, <POLYGON ((297.64 107.193, 297.64 131.068, 217.824 131.068, 217.824 107.193,...>, <POLYGON ((377.456 107.193, 377.456 131.068, 297.64 131.068, 297.64 107.193,...>, <POLYGON ((457.271 107.193, 457.271 131.068, 377.456 131.068, 377.456 107.19...>, <POLYGON ((537.087 107.193, 537.087 131.068, 457.271 131.068, 457.271 107.19...>, <POLYGON ((138.009 131.068, 138.009 154.322, 58.193 154.322, 58.193 131.068,...>, <POLYGON ((138.009 154.322, 138.009 177.369, 58.193 177.369, 58.193 154.322,...>, <POLYGON ((457.271 154.322, 457.271 223.464, 377.456 223.464, 377.456 154.32...>, <POLYGON ((537.087 154.322, 537.087 223.464, 457.271 223.464, 457.271 154.32...>, <POLYGON ((138.009 177.369, 138.009 200.417, 58.193 200.417, 58.193 177.369,...>, <POLYGON ((138.009 200.417, 138.009 223.464, 58.

In [21]:
s = [1,2,3,4,5]
s.pop(0)
print(s)

[2, 3, 4, 5]


In [14]:
for i in pdf_document:
    print(i.get_drawings())

[{'items': [('re', Rect(0.0, 0.0, 596.0, 842.0), -1)], 'type': 'f', 'even_odd': False, 'fill_opacity': 1.0, 'fill': (1.0, 1.0, 1.0), 'rect': Rect(0.0, 0.0, 596.0, 842.0), 'seqno': 0, 'layer': '', 'closePath': None, 'color': None, 'width': None, 'lineCap': None, 'lineJoin': None, 'dashes': None, 'stroke_opacity': None}, {'items': [('re', Rect(58.19292068481445, 107.19293212890625, 138.00860595703125, 131.06793212890625), -1)], 'type': 'f', 'even_odd': False, 'fill_opacity': 1.0, 'fill': (0.7434958219528198, 0.7536888718605042, 0.7505760192871094), 'rect': Rect(58.19292068481445, 107.19293212890625, 138.00860595703125, 131.06793212890625), 'seqno': 5, 'layer': '', 'closePath': None, 'color': None, 'width': None, 'lineCap': None, 'lineJoin': None, 'dashes': None, 'stroke_opacity': None}, {'items': [('re', Rect(138.00860595703125, 107.19293212890625, 217.8242950439453, 131.06793212890625), -1)], 'type': 'f', 'even_odd': False, 'fill_opacity': 1.0, 'fill': (0.7434958219528198, 0.75368887186

In [11]:

for i in pdf_document:
    drawings = i.get_drawings()
    is_short_line = lambda x: abs(x['rect'][3] - x['rect'][1]) < 10 and abs(x['rect'][2] - x['rect'][0]) < 10
    drawings = [drawing for drawing in drawings if not is_short_line(drawing)]

    # 转换为shapely的矩形
    rect_list = [sg.box(*drawing['rect']) for drawing in drawings]

    # 提取图片区域
    images = i.get_image_info()
    image_rects = [sg.box(*image['bbox']) for image in images]

    # 合并drawings和images
    rect_list += image_rects

In [12]:
rect_list

[]

In [3]:
pdf_document

Document('課表.pdf')

In [6]:
import pymupdf4llm
res = pymupdf4llm.to_markdown('testv2.pdf',page_chunks=True,margins=(0,0,0,0))

In [7]:
print(res[0]['text'])

在這個錯綜複雜的世界裡，我們常常會發現自己被各種無法言說的情感所困擾，無論是從宏觀角度
來看，還是從微觀層面來分析，這些情感似乎無處不在。隨著時間的推移，我們對於這些情感的理
解也許會變得更加深刻，但也可能會陷入一種莫名其妙的迷茫之中。無論如何，我們都應該時刻保
持一顆開放的心靈，去接受那些可能從未預料到的驚喜和挑戰。這樣一來，我們才能在這個變幻莫
測的社會中找到一絲絲的平靜。然而，這些平靜又往往是短暫的，因為在這個世界上，唯一不變的
就是變化本身。我們的思維就像一條蜿蜒曲折的河流，時而平靜，時而洶湧，但無論如何，它最終
都會流向那未知的遠方。這種不確定性雖然令人不安，但同時也賦予了我們生命的無限可能性。在
這樣的背景下，我們所能做的，就是盡量保持內心的平衡，去迎接那不可預知的未來。

|Col1|星期一|星期二|星期三|
|---|---|---|---|
|Vincent|Work|||
|Nils||Work||
|Mia||Work||
|Sean|Work||Work|



在金融業的生活充滿了緊張和刺激，無論是站在交易所的地板上，還是在高樓林立的辦公室內，總
有一種不可言喻的壓力時刻伴隨著你。每天早晨，金融從業者們都會在日出之前起床，手握咖啡，
匆忙地閱讀全球市場的最新動向，試圖在一片數據的海洋中找到一些價值資訊。隨著市場的開盤，
辦公室內電話聲此起彼伏，數據圖表在屏幕上不停跳動，交易的決策往往在幾秒鐘內做出，而這些
決策可能會影響到數百萬甚至數千萬的資金。
這種高壓的工作環境要求金融從業者們擁有極高的專業素養和心理素質，他們必須時刻保持頭腦清
醒，應對瞬息萬變的市場情況。同時，為了應對長時間的高強度工作，他們也會尋找各種方式來舒
緩壓力，無論是健身、冥想還是簡單的放鬆活動。儘管金融業的生活可能讓人感到疲憊不堪，但成
功達成一筆交易或看到市場走向如自己所預期時，帶來的那份成就感和滿足感，卻是難以言喻的。
這是一個充滿挑戰的領域，但也正因如此，才讓人無法抗拒其中的魅力。

|2023 國泰世華人員名單|Col2|Col3|Col4|Col5|Col6|
|---|---|---|---|---|---|
|姓名|職位|學校|科系|年級|聯絡方式|
|Vincent Hsia|Cip實習|NCCU|MIS|一|0912-3456756|
|Mia Chen

In [2]:
import pymupdf4llm
res = pymupdf4llm.to_markdown('113Q1.pdf',page_chunks=True)